# Imports and Loading Data

In [145]:
# importing necessary libraries
import pandas as pd
import sqlite3

In [146]:
# loading orders data (CSV)
orders = pd.read_csv("orders.csv")
orders.head(2)

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine


The orders dataset contains transactional information where each row
represents a single order.

In [147]:
# loading users data (JSON)
users = pd.read_json("users.json")
users.head(2)

,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold


The users dataset contains user details such as city and membership type.

In [148]:
# loading restaurants data (SQL)

# create temp SQLite database in memory
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

with open("restaurants.sql", "r") as f:
    sql_script = f.read()

# execute SQL script
cursor.executescript(sql_script)

# load restaurants table
restaurants = pd.read_sql("SELECT * FROM restaurants", conn)

restaurants.head(2)

,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1


The restaurant data is provided in SQL format and contains restaurant information such as cuisine type and rating.

In [149]:
orders.info()
users.info()
restaurants.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   order_id         10000 non-null  int64  
 1   user_id          10000 non-null  int64  
 2   restaurant_id    10000 non-null  int64  
 3   order_date       10000 non-null  object 
 4   total_amount     10000 non-null  float64
 5   restaurant_name  10000 non-null  object 
dtypes: float64(1), int64(3), object(2)
memory usage: 468.9+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     3000 non-null   int64 
 1   name        3000 non-null   object
 2   city        3000 non-null   object
 3   membership  3000 non-null   object
dtypes: int64(1), object(3)
memory usage: 93.9+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0

# Final Dataset Preparation

In [150]:
# Before merging duplicate columns are handled to avoid confusion

# Drop restaurant_name from restaurants to avoid duplication after merge
restaurants = restaurants.drop(columns=["restaurant_name"])

In [151]:
# Merging Datasets

df = orders.merge(users, on="user_id", how="left")
df = df.merge(restaurants, on="restaurant_id", how="left")

df.head(2)

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name,name,city,membership,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Indian,4.5


In [152]:
df.shape

(10000, 11)

In [153]:
# Final Dataset

try:
    df.to_csv("final_food_delivery_dataset.csv", index=False)
    print("CSV file saved successfully as final_food_delivery_dataset.csv")
except Exception as e:
    print("Error while saving CSV file:", e)

CSV file saved successfully as final_food_delivery_dataset.csv


# Data Analysis and Insights

The following analysis answers the questions provided in the hackathon

In [154]:
# Which city has the highest total revenue (total_amount) from Gold members?

top_gold_city = (
    df[df
    ["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .sum()
    .sort_values(ascending=False)
    .head(1)
)
top_gold_city

,total_amount
city,
Chennai,1080909.79


In [155]:
# Which cuisine has the highest average order value across all orders?

top_cuisine_hig_avg = (
    df
    .groupby("cuisine")["total_amount"]
    .mean()
    .sort_values(ascending=False)
    .head(1)
)

top_cuisine_hig_avg

,total_amount
cuisine,
Mexican,808.021344


In [156]:
# How many distinct users placed orders worth more than ₹1000 in total (sum of all their orders)?

users_order_above_1000 = (
    df
    .groupby("user_id")["total_amount"]
    .sum()
)

users_order_above_1000 = users_order_above_1000[users_order_above_1000 > 1000].count()

users_order_above_1000

np.int64(2544)

In [157]:
# Which restaurant rating range generated the highest total revenue?

top_rating_range = (
    df
    .groupby("rating")["total_amount"]
    .sum()
    .sort_values(ascending=False)
    .head(1)
)

top_rating_range

,total_amount
rating,
4.8,657707.71


In [158]:
# Among Gold members, which city has the highest average order value?

top_gold_city_avg = (
    df[df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .mean()
    .sort_values(ascending=False)
    .head(1)
)

top_gold_city_avg

,total_amount
city,
Chennai,808.45908


In [159]:
# Which cuisine has the lowest number of distinct restaurants but still contributes significant revenue?

cuisine_analysis = (
    df
    .groupby("cuisine")
    .agg({
        "restaurant_id": "nunique",
        "total_amount": "sum"
    })
    .sort_values(["restaurant_id", "total_amount"], ascending=[True, False])
    .head(1)
)

cuisine_analysis

,restaurant_id,total_amount
cuisine,,
Chinese,120,1930504.65


In [160]:
# Which restaurant has the highest average order value but less than 20 total orders?
# Among this (Grand Cafe Punjabi, Grand Restaurant South Indian, Ruchi Mess Multicuisine, Ruchi Foods Chinese)?

restaurant_options = [
    "Grand Cafe Punjabi",
    "Grand Restaurant South Indian",
    "Ruchi Mess Multicuisine",
    "Ruchi Foods Chinese"
]

top_restaurant_high_avg_low_orders = (
    df[df["restaurant_name"].isin(restaurant_options)]
    .groupby("restaurant_name")
    .agg(
        avg_order_value=("total_amount", "mean"),
        total_orders=("order_id", "count")
    )
    .query("total_orders < 20")
    .sort_values(by="avg_order_value", ascending=False)
    .head()
)

top_restaurant_high_avg_low_orders

,avg_order_value,total_orders
restaurant_name,,
Ruchi Foods Chinese,686.603158,19


In [161]:
# Which combination contributes the highest revenue?

top_membership_cuisine_combo = (
    df
    .groupby(["membership", "cuisine"])["total_amount"]
    .sum()
    .sort_values(ascending=False)
    .head()
)

top_membership_cuisine_combo

membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Name: total_amount, dtype: float64

In [162]:
# During which quarter of the year is the total revenue highest?

# convert order_date to datetime
df["order_date"] = pd.to_datetime(df["order_date"], dayfirst=True)

# create quarter column
df["quarter"] = df["order_date"].dt.to_period("Q")

top_revenue_quarter = (
    df
    .groupby("quarter")["total_amount"]
    .sum()
    .sort_values(ascending=False)
    .head(1)
)

top_revenue_quarter

,total_amount
quarter,
2023Q3,2037385.1


In [163]:
# How many total orders were placed by users with Gold membership?

total_gold_orders = (
    df[df["membership"] == "Gold"]
    .shape[0]
)

total_gold_orders

4987

In [164]:
# What is the total revenue (rounded to nearest integer) generated from orders placed in Hyderabad city?

hyderabad_total_revenue = (
    df[df["city"] == "Hyderabad"]["total_amount"]
    .sum()
)

round(hyderabad_total_revenue)

1889367

In [165]:
# How many distinct users placed at least one order?

distinct_users_count = df["user_id"].nunique()
distinct_users_count

2883

In [166]:
# What is the average order value (rounded to 2 decimals) for Gold members?

gold_avg_order_value = (
    df[df["membership"] == "Gold"]["total_amount"]
    .mean()
)

round(gold_avg_order_value, 2)

np.float64(797.15)

In [167]:
# How many orders were placed for restaurants with rating ≥ 4.5?

orders_high_rating = (
    df[df["rating"] >= 4.5]
    .shape[0]
)

orders_high_rating

3374

In [168]:
# How many orders were placed in the top revenue city among Gold members only?

# find the top revenue city among Gold members
top_gold_city = (
    df[df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .sum()
    .sort_values(ascending=False)
    .head(1)
    .index[0]
)

# count orders in that city
orders_in_top_gold_city = (
    df[
        (df["membership"] == "Gold") &
        (df["city"] == top_gold_city)
    ]
    .shape[0]
)

orders_in_top_gold_city

1337